In [ ]:
import globus_automate_client
import mdf_toolbox
import json
from copy import deepcopy

In [ ]:
native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"  # Premade native app ID (from CFDE)
flows_client = globus_automate_client.create_flows_client(native_app_id)

In [ ]:
from getpass import getpass
smtp_user = getpass("SMTP Username: ")
smtp_pass = getpass("SMTP Password: ")
smtp_hostname = "email-smtp.us-east-1.amazonaws.com"

In [ ]:
#flow_permissions = ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"]  # jgaff@uchicago.edu
flow_permissions = ["urn:globus:groups:id:5fc63928-3752-11e8-9c6f-0e00fd09bf20"]  # MDF Connect Admins
admin_permissions = flow_permissions
# admin_permissions = ["urn:globus:groups:"]
curation_subflow_url = "http://flows.automate.globus.org/flows/a24d39dd-f8b7-4287-ba4d-cdd8e36fcee6"
curation_subflow_scope = "https://auth.globus.org/scopes/a24d39dd-f8b7-4287-ba4d-cdd8e36fcee6/flow_a24d39dd_f8b7_4287_ba4d_cdd8e36fcee6"
transfer_loop_url = "https://flows.globus.org/flows/5efa85a0-54d7-426e-b9f1-59452fda9922"
transfer_loop_scope = "https://auth.globus.org/scopes/5efa85a0-54d7-426e-b9f1-59452fda9922/flow_5efa85a0_54d7_426e_b9f1_59452fda9922_user"

admin_email = "jgaff@uchicago.edu"
sender_email = "materialsdatafacility@gmail.com"

In [ ]:
mock_dataset_entry = {'dc': {'titles': [{'title': 'Base Deploy Testing Dataset'}],
  'creators': [{'creatorName': 'jgaff',
    'familyName': '',
    'givenName': 'jgaff',
    'affiliations': ['UChicago']}],
  'publisher': 'Materials Data Facility',
  'publicationYear': '2020',
  'resourceType': {'resourceTypeGeneral': 'Dataset',
   'resourceType': 'Dataset'}},
 'mdf': {'source_id': '_test_base_deploy_testing_v5.1',
  'source_name': '_test_base_deploy_testing',
  'version': 5,
  'acl': ['public'],
  'scroll_id': 0,
  'ingest_date': '2020-05-06T17:47:05.219450Z',
  'resource_type': 'dataset'},
 'data': {'endpoint_path': 'globus://e38ee745-6d04-11e5-ba46-22000b92c6ec/MDF/mdf_connect/prod/data/_test_base_deploy_testing_v5.1/',
  'link': 'https://app.globus.org/file-manager?origin_id=e38ee745-6d04-11e5-ba46-22000b92c6ec&origin_path=/MDF/mdf_connect/prod/data/_test_base_deploy_testing_v5.1/',
  'total_size': 4709193},
 'services': {}}


In [ ]:
transfer_input_schema = {
    # "deadline": "datetime str",
    "destination_endpoint_id": "str",
    "label": "str",
    "source_endpoint_id": "str",
    # "sync_level": "str 0-3",
    "transfer_items": [{
        "destination_path": "str",
        "recursive": "bool",
        "source_path": "str"
    }]
}
transfer_permission_schema = {
    "endpoint_id": "string",
    "operation": "string",
    "path": "string",
    "permissions": "string",
    "principal": "string",
    "principal_type": "string"
}
curation_input_schema = {
    "curator_emails": "list of str, or False",
    "curator_template": "str or False",  # variables: $landing_page
    "curation_permissions": "list of str",
    "curation_text": "str or False",
    "author_email": "str or False",
    "author_template": "str or False",  # variables: $curation_task_id, $decision, $reason
    "email_sender": "str",
    "send_credentials": [{}]
}
xtract_input_schema = {
    "metadata_storage_ep": "str",
    "eid": "str",
    "dir_path": "str",
    "mapping": "match",  # ?
    "dataset_mdata": {"test1": "test2"},
    "validator_params": {"schema_branch": "master", "validation_info": {"test1": "test2"}},
    "grouper": "matio"  # options are 'directory/matio'
}

In [ ]:
mdf_flow_def = {
    "title": "The Materials Data Facility Dataset Processing Flow",
    "description": "Extract, process, and ingest a dataset into MDF Connect.",
    "visible_to": flow_permissions,
    "runnable_by": flow_permissions,
    "administered_by": admin_permissions,
    "definition": {
        "StartAt": "StartSubmission",
        "States": {
            # Can add start email here if desired
            "StartSubmission": {
                "Type": "Pass",
                "Next": "UserPermissions"
            },
            # Grant user write permissions on MDF storage (temporarily)
            "UserPermissions": {
                "Type": "Pass",
                #"Type": "Action",
                #"ActionUrl": "https://actions.globus.org/transfer/set_permission",
                #"ExceptionOnActionFailure": True,
                "Parameters": {
                #    "endpoint_id.$": "$.mdf_storage_ep",
                #    "operation": "CREATE",
                #    "path.$": "$.mdf_dataset_path",
                #    "permissions": "rw",
                #    "principal.$": "$.user_id",
                #    "principal_type": "identity"
                },
                "ResultPath": "$.UserPermissionResult",
                #"WaitTime": 86400,
                "Next": "UserTransfer"
            },
             # There must be at least one data location
            "UserTransfer": {
                "Type": "Action",
                "ActionUrl": transfer_loop_url,
                "ActionScope": transfer_loop_scope,
                "ExceptionOnActionFailure": False,
                #"RunAs": "MDFUser",
                "Parameters": {
                    "action_inputs.$": "$.user_transfer_inputs"
                },
                "ResultPath": "$.UserTransferResult",
                "WaitTime": 86400,
                "Next": "UndoUserPermissions"
            },
            "UndoUserPermissions": {
                "Type": "Pass",
#                "Type": "Action",
#                "ActionUrl": "https://actions.globus.org/transfer/set_permission",
#                "ExceptionOnActionFailure": True
                "Parameters": {},
#                    # TODO: Add when feature available
#                },
                "ResultPath": "$.UndoUserPermissionResult",
#                "WaitTime": 86400,
                "Next": "CheckUserTransfer"
            },
            "CheckUserTransfer": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.UserTransferResult.status",
                    "StringEquals": "SUCCEEDED",
                    "Next": "Xtraction"
                }],
                "Default": "FailUserTransfer"
            },
            "FailUserTransfer": {
                "Type": "ExpressionEval",
                "Parameters": {
                    "title": "MDF Submission Failed",
                    # Some errors listed in .details, others in .details.event_list.details.error.body
                    "message.=": ("'Your MDF submission ' + `$.source_id` + ' failed to transfer to MDF:\n'"
                                  " + `$.UserTransferResult.details`")
                },
                "ResultPath": "$.FinalState",
                "Next": "ChooseNotifyUserEnd"
            },
            "Xtraction": {
                "Type": "Pass",
#                "Type": "Action",
#                "ActionUrl": "https://xtract.materialsdatafacility.org/",
#                #"ActionScope": "https://auth.globus.org/scopes/34284fb1-2eea-4532-a04a-9c8ad1702856/xtract_crawl_and_extract",
#                "ExceptionOnActionFailure": True,
                "Parameters": {
#                    "metadata_storage_ep.$": "$.mdf_storage_ep",
#                    "eid.$": "$.mdf_storage_ep",
#                    "dir_path.$": "$.mdf_dataset_path",
#                    "mapping": "match",  # ?
#                    "dataset_mdata.$": "$.dataset_mdata",
#                    "validator_params.$": "$.validator_params",
#                    # options are 'directory/matio'
#                    "grouper.=": "'directory' if `$.group_by_dir` else 'matio'"
                    "details": {
                        "output_link": "https://e38ee745-6d04-11e5-ba46-22000b92c6ec.e.globus.org/MDF/mdf_connect/test_files/mock_feedstock.json",
                        "dataset_entry": mock_dataset_entry
                    }
                },
#                "Catch": [{
#                    "ErrorEquals": ["ActionFailedException"],
#                    "Next": "XtractionFail"
#                }, {
#                    "ErrorEquals": ["States.ALL"],
#                    "Next": "ExceptionState"
#                }],
                "ResultPath": "$.XtractionResult",
#                "WaitTime": 86400,
                "Next": "ChooseCuration"
            },
# Re-enable all Xtract together
#            "XtractionFail": {
#                "Type": "ExpressionEval",
#                "Parameters": {
#                    "title": "MDF Submission Failed",
#                    "message.=": ("Your MDF submission `$.source_id` failed during metadata extraction:\n"
#                                  "`$.XtractionResult.details`")
#                },
#                "ResultPath": "$.FinalState",
#                "Next": "ChooseNotifyUserEnd"
#            },
            "ChooseCuration": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.curation_input",
                    "BooleanEquals": False,
                    "Next": "SearchIngest"
                }],
                "Default": "CurateSubmission"
            },
            "CurateSubmission": {
                "Type": "Action",
                "ActionUrl": curation_subflow_url,
                "ActionScope": curation_subflow_scope,
                "ExceptionOnActionFailure": True,
                "InputPath": "$.curation_input",
                #"__Private_Parameters": ["send_credentials"],
                "ResultPath": "$.CurateResult",
                "WaitTime": 86400,
                "Next": "ChooseAcceptance"
            },
            "ChooseAcceptance": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.CurateResult.details.output.CurationResult.details.name",
                    "StringEquals": "accepted",
                    "Next": "SearchIngest"
                }, {
                    "Variable": "$.CurateResult.details.output.CurationResult.details.name",
                    "StringEquals": "rejected",
                    "Next": "FailCuration"
                }],
                "Default": "ExceptionState"
            },
            "SearchIngest": {
                "Type": "Action",
                "ActionUrl": "https://siap.globuscs.info/",
                "ActionScope": "https://auth.globus.org/scopes/a9b4124f-887a-461e-ba72-fa8ea701a8f2/siap_ingest_scope",
                "ExceptionOnActionFailure": True,                
                "Parameters": {
                    "auth_header.$": "$.feedstock_auth_header",
                    "index.$": "$.search_index",
                    "locations.=": "[`$.XtractionResult.details.output_link`]",
                    "require_all_success": True,
                    "__Private_Parameters": ["auth_header"]
                },
                "ResultPath": "$.SearchIngestResult",
                "WaitTime": 86400,
                "Next": "DataDestTransfer"
            },
            "DataDestTransfer":{
                "Type": "Action",
                "ActionUrl": transfer_loop_url,
                "ActionScope": transfer_loop_scope,
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "action_inputs.$": "$.data_destinations"
                },
                "ResultPath": "$.DataDestResult",
                "WaitTime": 86400,
                "Next": "ChoosePublish"
            },
            "ChoosePublish": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.mdf_publish",
                    "BooleanEquals": True,
                    "Next": "MDFPublish"
                }],
                "Default": "ChooseCitrine"
            },
            "MDFPublish": {
                # TODO: Enable when DCAP accepts "standard" DC schema
                "Type": "Pass",
                #"ActionUrl": "https://actions.globus.org/datacite/mint/basic_auth",
                #"ExceptionOnActionFailure": False,
                "Parameters": {
                },
                "ResultPath": "$.MDFPublishResult",
                #"WaitTime": 86400,
                "Next": "ChooseCitrine"
            },
            "ChooseCitrine": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.citrine",
                    "BooleanEquals": True,
                    "Next": "CitrinePublish"
                }],
                "Default": "ChooseMRR"
            },
            "CitrinePublish": {
                # TODO: FuncX function to publish to Citrine
                "Type": "Pass",
                #"ActionUrl": "",
                #"ExceptionOnActionFailure": False,
                "Parameters": {
                },
                "ResultPath": "$.CitrinePublishResult",
                #"WaitTime": 86400,
                "Next": "ChooseMRR"
            },
            "ChooseMRR": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.mrr",
                    "BooleanEquals": True,
                    "Next": "MRRPublish"
                }],
                "Default": "PrepareSearchUpdate"
            },
            "MRRPublish":{
                # TODO: FuncX function to publish to MRR
                "Type": "Pass",
                #"ActionUrl": "",
                #"ExceptionOnActionFailure": False,
                "Parameters": {
                },
                "ResultPath": "$.MRRPublishResult",
                #"WaitTime": 86400,
                "Next": "PrepareSearchUpdate"
            },
            "PrepareSearchUpdate": {
                "Type": "ExpressionEval",
                # TODO: Apply services changes to dataset entry
                "Parameters":{
                    "subject.$": "$.source_id",
                    "content.=": "`$.XtractionResult.details.dataset_entry`",
                    "visible_to.$": "$.dataset_acl",
                    "search_index.$": "$.search_index"
                },
                "ResultPath": "$.SearchUpdateInfo",
                "Next": "SearchUpdate"
            },
            "SearchUpdate": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/search/ingest",
                #"ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/search/ingest",
                "ExceptionOnActionFailure": False,
                "InputPath": "$.SearchUpdateInfo",
                "ResultPath": "$.SearchUpdateResult",
                "WaitTime": 86400,
                "Next": "SubmissionSuccess"
            },
            "SubmissionSuccess": {
                "Type": "ExpressionEval",
                "Parameters": {
                    "title": "Submission Ingested Successfully",
                    "message.=": ("'Submission Flow succeeded. Your submission (' + `$.source_id`"
                                  "+ ') can be viewed at this link: ' + `$.mdf_portal_link`")
                },
                "ResultPath": "$.FinalState",
                "Next": "ChooseNotifyUserEnd"
            },
            "FailCuration": {
                "Type": "ExpressionEval",
                "Parameters": {
                    "title": "MDF Submission Rejected",
                    "message.=": ("'Your submission (' + `$.source_id` + ') was rejected by a curator "
                                  "and did not complete the ingestion process. The curator gave the "
                                  "following reason for rejection: '"
                                  "+ `$.CurateResult.details.output.CurationResult.details.parameters.user_input`")
                },
                "ResultPath": "$.FinalState",
                "Next": "ChooseNotifyUserEnd"
            },
            "ExceptionState": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                #"ActionScope": "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template.=": ("'Submission ' + `$.source_id` + ' fatally errored processing in Flow '"
                                        "+ `$._context.action_id` + '. Please review the Flow log for details "
                                        "about this exception.'"),
                    "destination": admin_email,
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"],
                    "sender": sender_email,
                    "subject": "Submission Failed to Ingest"
                },
                "ResultPath": "$.ExceptionNotifyResult",
                "WaitTime": 86400,
                "Next": "SubmissionException"
            },
            "SubmissionException": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                #"ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "title": "Service Error in MDF Submission",
                    "message": ("A service error has occurred, and the MDF team has been notified. "
                                "You may be contacted with additional details.")
                },
                "ResultPath": "$.FinalState",
                "WaitTime": 86400,
                "Next": "ChooseNotifyUserEnd"
            },
            "ChooseNotifyUserEnd": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.curation_input",
                    "BooleanEquals": False,
                    "Next": "EndSubmission"
                }],
                "Default": "NotifyUserEnd"
            },
            "NotifyUserEnd":{
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                #"ActionScope": "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template.$": "$.FinalState.message",
                    "destination.$": "$.curation_input.author_email",
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"],
                    "sender": sender_email,
                    "subject.$": "$.FinalState.title"
                },
                "ResultPath": "$.NotifyUserResult",
                "WaitTime": 86400,
                "Next": "EndSubmission"
            },
            "EndSubmission": {
                "Type": "Pass",
                "End": True
            }
        }
    },
    "schema": {
        "source_id": "str",
        "user_id": "str",
        "mdf_portal_link": "str, must be complete link for after submission succeeds",

        # Data sources
        "user_transfer_sources": [{
            "ep": "UUID",
            "path": "str"
        }],
        "data_destinations": [{  # Can be empty []
            "ep": "UUID",
            "path": "str"
        }],
        "file_acls": ["FQ UUID"],

        "dataset_acl": ["FQ UUID"],
        "search_index": "UUID",

        "group_by_dir": "bool",
        "mdf_storage_ep": "str",
        "mdf_dataset_path": "str",
        "dataset_mdata": "dict",
        "validator_params": "dict",

        "feedstock_https_domain": "str",
        "feedstock_auth_header": "str",

        "curation_input": curation_input_schema,  # or False
        "mdf_publish": "bool",
        "citrine": "bool",
        "mrr": "bool",

        "_tokens": {
            "User": "user's token"
        }
    }
}

In [ ]:
"""
flow_deploy_res = flows_client.deploy_flow(
    flow_definition=mdf_flow_def["definition"],
    title=mdf_flow_def["title"],
    description=mdf_flow_def["description"],
    visible_to=mdf_flow_def["visible_to"],
    runnable_by=mdf_flow_def["runnable_by"],
    administered_by=mdf_flow_def["administered_by"],
    # TODO
    input_schema={},
    validate_definition=True,
    validate_input_schema=True
)
flow_id = flow_deploy_res["id"]
flow_scope = flow_deploy_res["globus_auth_scope"]
print(flow_deploy_res)
"""
flow_update_res = flows_client.update_flow(
    flow_id,
    flow_definition=mdf_flow_def["definition"],
    title=mdf_flow_def["title"],
    description=mdf_flow_def["description"],
    visible_to=mdf_flow_def["visible_to"],
    runnable_by=mdf_flow_def["runnable_by"],
    administered_by=mdf_flow_def["administered_by"],
    # TODO
    input_schema={},
    validate_definition=True,
    validate_input_schema=True)

with open("mdf_flow_definition.json", 'w') as f:
    json.dump(mdf_flow_def, f, indent=4, sort_keys=True)

In [ ]:

print(flow_id)
print(flow_scope)


In [ ]:
'''
logins = mdf_toolbox.login(services=["petrel", flow_scope], make_clients=False)
feedstock_auth_header = {}
run_as_token = {}
logins["petrel"].set_authorization_header(feedstock_auth_header)
feedstock_auth_header = feedstock_auth_header["Authorization"]
run_as_token = logins[flow_scope].refresh_token
'''
'''
logins[flow_scope].set_authorization_header(run_as_token)
run_as_token = run_as_token["Authorization"].split(" ")[1]
'''

In [ ]:
import time
test_time = int(time.time())
test_user_id = "urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"  # jgaff@uchicago.edu
test_index = mdf_toolbox.translate_index("mdf-dev")
mdf_ep = "e38ee745-6d04-11e5-ba46-22000b92c6ec"
mdf_data_path = "/MDF/mdf_connect/test_files/deleteme/data/test123/"
mdf_feedstock_dir = "/MDF/mdf_connect/test_files/deleteme/feedstock/"
mdf_ep_https_domain = "https://e38ee745-6d04-11e5-ba46-22000b92c6ec.e.globus.org"

source_ep = "e38ee745-6d04-11e5-ba46-22000b92c6ec"
source_path = "/MDF/mdf_connect/test_files/canonical_datasets/dft/"

user_transfers = [{
    "destination_endpoint_id": mdf_ep,
    "label": "MDF Flow Test Transfer1",
    "source_endpoint_id": source_ep,
    "transfer_items": [{
        "destination_path": mdf_data_path,
        "recursive": True,
        "source_path": source_path
    }]
}]
data_dests = [{
}]
data_permissions = {
}
curation_input = False
test_dataset_mdata = {}
test_validator_params = {}

flow_input = {
    "source_id": f"test_{test_time}_mdf",
    "user_id": test_user_id,
    "mdf_portal_link": f"https://example.com/{test_time}",
    "user_transfer_inputs": user_transfers,
    "data_destinations": [], #data_dests,
    "data_permissions": data_permissions,

    "dataset_acl": [test_user_id],
    "search_index": test_index,

    "group_by_dir": False,
    "mdf_storage_ep": mdf_ep,
    "mdf_dataset_path": mdf_data_path,
    "dataset_mdata": test_dataset_mdata,
    "validator_params": test_validator_params,

    "feedstock_ep": mdf_ep,
    "feedstock_dir": mdf_feedstock_dir,
    "feedstock_https_domain": mdf_ep_https_domain,
    "feedstock_auth_header": feedstock_auth_header,

    "curation_input": curation_input,  # or False
    "mdf_publish": False,
    "citrine": False,
    "mrr": False,
    
#    "_tokens": {
#        "User": run_as_token
#    }
}
#print(json.dumps(user_transfers, indent=2, sort_keys=True))
with open("mdf_flow_sample_input.json", 'w') as f:
    dumpable_input = 
    json.dump(flow_input, f, indent=4, sort_keys=True)

In [ ]:
flow_res = flows_client.run_flow(flow_id, flow_scope, flow_input)
flow_res.data

In [ ]:
status = flows_client.flow_action_status(flow_id, flow_scope, flow_res["action_id"]).data
print(json.dumps(status, indent=4, sort_keys=True))

In [ ]:
flows_client.flow_action_log(flow_id, flow_scope, flow_res["action_id"], limit=100).data

In [ ]:
flows_client.deploy_flow?